In [62]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random
import json

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import os
os.chdir('drive/My Drive/Colab Notebooks/data')

FileNotFoundError: ignored

In [0]:
ls

In [0]:
with open('intents.json') as file :
  data = json.load(file)

print(data['intents'])

In [0]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents'] :
  for pattern in intent["patterns"] :
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)

    docs_x.append(wrds)
    docs_y.append(intent['tag'])

  if intent["tag"] not in labels :
    labels.append(intent['tag'])


In [0]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

In [0]:
labels = sorted(labels)

In [0]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x) :
  bag = []

  wrds = [stemmer.stem(w) for w in doc]

  for w in words :
    if w in wrds :
      bag.append(1)
    else :
      bag.append(0)
  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] =1

  training.append(bag)
  output.append(output_row)

training = numpy.array(training)
output = np.array(output)



with open("data.pickle", "wb") as f :
  pickle.dump((words, labels, training, output),f)

In [61]:
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [64]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation= 'softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

try : 
  model.load("model.tflearn")
except :
  model.fit(training, output, n_epoch=1000, batch_size = 8, show_metric=True)
  model.save('model.tflearn')


Training Step: 3999  | total loss: 0.05701 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.05701 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.05623 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.05623 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:/content/drive/My Drive/Colab Notebooks/data/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [66]:
import pickle
with open("data.pickle","rb") as f :
  words, labels, training, ouput = pickle.load(f)


FileNotFoundError: ignored

In [0]:
def bag_of_words(s, words) :
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words :
    for i,w in enumerate(words) :
      if w == se :
        bag[i] = 1

  return numpy.array(bag)

def chat() :
  print("start talking with the bot! \n")
  print("type quit to stop")
  while True :
    inp = input("You : ")
    if inp.lower() == "quit" :
      break
    result =  np.argmax(model.predict([bag_of_words(inp, words)]))
    tag = labels[result]
    

    for tg in data["intents"] :
      if tg['tag'] == tag :
        responses = tg['responses']

    print(random.choice(responses))

In [79]:
chat()

start talking with the bot! 

type quit to stop
You : cya
Goodbye!
You : hi
Good to see you again!
You : hello
Good to see you again!
You : good bye
Good to see you again!
You : sell
Hi there, how can I help?
You : quit
